## Config Iniciais

In [ ]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install torch

In [ ]:
import os
import transformers
import torch
import getpass
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline


In [ ]:
login(token="_token_")
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
)
                      


In [ ]:
text_generator = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.1,
)


def get_response(prompt):
  response = text_generator(prompt)
  gen_text = response[0]['generated_text']
  return gen_text

### Exemplos

In [ ]:
prompt = "O que é computação quântica? desejo que a minha resposta venha em portugues"
llama_response = get_response(prompt)
print(llama_response)


# prompt = "Qual o resultado de 7 x 6 - 42?"
# llama_response = get_response(prompt)
# print(llama_response)

# prompt = "Quem foi a primeira pessoa no espaço?"
# llama_response = get_response(prompt)
# print(llama_response)

## templates e Engenharias de Prompt

In [ ]:
def get_response(prompt):
  response = text_generator(prompt)
  gen_text = response[0]['generated_text']
  return gen_text

In [ ]:
prompt = "Quem foi a primeira pessoa no espaço?"
template  = """<|system|>
você é um assistente útil.

<|user|>
"{}"<|end|>

<|assistant|>
""".format(prompt)
output = get_response(template)
print(output)

In [ ]:
prompt = "Você entende português?"
template  = """<|system|>
você é um assistente útil.

<|user|>
"{}"<|end|>

<|assistant|>
""".format(prompt)
output = get_response(template)
print(output)

In [ ]:
prompt = "O que é ia" #@param {type:"string"}

template  = """<|system|>
você é um assistente útil.

<|user|>
"{}"<|end|>

<|assistant|>
""".format(prompt)
output = get_response(template)
print(output)

## Explorando Engenharia de Prompt

In [ ]:
# prompt = "O que é ia" #@param {type:"string"}
# prompt = "O que é IA? responda em 1 frase"
prompt = "O que é IA? responda em forma de poema"
sys_prompt = "Você é um assistente virtual prestativo, responda as perguntas em português!"
template  = """<|system|>
{}<|end|>

<|user|>
"{}"<|end|>

<|assistant|>
""".format(sys_prompt , prompt)
output = get_response(template)
print(output)

In [ ]:
prompt = "Gere um código em python que escreva a sequência de fibonacci"
sys_prompt = "Você é um programador experiente. Retorne o código requisitado e se quiser forneça explicações breves, caso convenientes"
template  = """<|system|>
{}<|end|>

<|user|>
"{}"<|end|>

<|assistant|>
""".format(sys_prompt , prompt)
output = get_response(template)
print(output)

## Formato de Mensagens

In [ ]:
prompt = "O que é IA?"

msg = [
    
       {"role" : "system" , "content" : "Você é um assistente virtual prestativo, responda as perguntas em português!"},
       {"role" : "user" , "content" : prompt}
]

output = get_response(msg)
print(output)

In [ ]:
prompt = "Liste o nome de 10 cidades famosas da Europa"

msg = [
    
       {"role" : "system" , "content" : "Você é um assistente virtual prestativo, responda as perguntas em português!"},
       {"role" : "user" , "content" : prompt}
]

output = get_response(msg)
print(output)

## Quantização - Bits and bytes config

### - A quantização reduz a precisão dos números usados para representar os parâmetros do modelo, diminuindo o número de memória e os requisitos computacionais

### - Ao invés de usar números de ponto flutuantes de 32 bits(float32), o modelo pode usar 16 , ou até 8 bits.

### - Este processo pode reduzir significativamente o tamanho do modelo e acelerar a inferência sem causar uma perda substancial na precisão


### Benefícios para LLMs:

1 . permite rodar grandes modelos de linguagem em hardware com recursos limitados;

2. Mantém um bom desempenho sem comprometer significativamente a precisão

## Usando o Meta-Llama-3-8B-instruct

In [ ]:
prompt = ("Quem foi a primeira pessoa no espaço?")
messages = [{"role" : "user" , "content" : prompt}]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
model_inputs = encodeds.to(device)
generate_ids = model.generate(
    model_inputs, 
    max_new_tokens=1000, 
    do_sample=True, 
    pad_token_id=tokenizer.pad_token_id,  
    eos_token_id=tokenizer.eos_token_id
)
decoded = tokenizer.batch_decode(generate_ids)

output = decoded[0]
print(output)

In [3]:
import requests
import zipfile

def baixar_extrair_transparencia(url, target_dir):
    """Baixa um arquivo ZIP de uma URL, extrai seu conteúdo e salva os arquivos com um sufixo.
    Args:
        url (str): URL do arquivo ZIP.
        target_dir (str): Diretório de destino para salvar os arquivos extraídos.
    """
    response = requests.get(url, allow_redirects=True, timeout=600)
    response.raise_for_status()

    content_type = response.headers.get("Content-Type", "")
    if (
        "application/zip" in content_type
        or "application/x-zip-compressed" in content_type
    ):
        file_name = url.split("/")[-1]
        zip_path = os.path.join(target_dir, file_name)

        with open(zip_path, "wb") as file:
            file.write(response.content)
        print(f"Arquivo {file_name} baixado e salvo em {target_dir}.")

        # Tenta extrair o arquivo ZIP
        try:
            with zipfile.ZipFile(zip_path, "r") as zip_ref:
                for member in zip_ref.namelist():
                    member_filename = os.path.basename(member)
                    new_filename = f"{file_name.replace('.zip', '')}_{member_filename}"
                    new_filepath = os.path.join(target_dir, new_filename)

                    with open(new_filepath, "wb") as new_file:
                        new_file.write(zip_ref.read(member))
            print(f"Arquivos de {file_name} extraídos com sucesso.")
        except zipfile.BadZipFile:
            print(f"Erro: O arquivo {file_name} não é um ZIP válido.")
    else:
        print(
            f"Erro: O conteúdo baixado de {url} não é um arquivo ZIP."
            "Tipo de conteúdo: {content_type}"
        )

In [13]:
import datetime
from dateutil.relativedelta import relativedelta


def baixar_dados_referencia():
    """Baixa e processa dados de referência do Portal da Transparência para uma data específica.
    Args:
        dat_ref (str): Data de referência no formato 'YYYYMM'.
    """
    dat_ref = 202501
    dat_ref = datetime.strptime(dat_ref, "%Y%m%d")
    dat_ref = dat_ref.strftime("%Y%m")
    print(f"Processando data de referência: {dat_ref}")

    # Dicionário com URLs para download do Portal da Transparência
    urls = {
        "Militares": (
            f"https://portaldatransparencia.gov.br/download-de-dados/servidores/"
            f"{dat_ref}_Militares"
        ),
        "Pensionistas_DEFESA": (
            f"https://portaldatransparencia.gov.br/download-de-dados/servidores/"
            f"{dat_ref}_Pensionistas_DEFESA"
        ),
        "Reserva_Reforma_Militares": (
            f"https://portaldatransparencia.gov.br/download-de-dados/servidores/"
            f"{dat_ref}_Reserva_Reforma_Militares"
        ),
        "Servidores_SIAPE": (
            f"https://portaldatransparencia.gov.br/download-de-dados/servidores/"
            f"{dat_ref}_Servidores_SIAPE"
        ),
        "Pensionistas_SIAPE": (
            f"https://portaldatransparencia.gov.br/download-de-dados/servidores/"
            f"{dat_ref}_Pensionistas_SIAPE"
        ),
        "Aposentados_SIAPE": (
            f"https://portaldatransparencia.gov.br/download-de-dados/servidores/"
            f"{dat_ref}_Aposentados_SIAPE"
        ),
        "Servidores_BACEN": (
            f"https://portaldatransparencia.gov.br/download-de-dados/servidores/"
            f"{dat_ref}_Servidores_BACEN"
        ),
        "Aposentados_BACEN": (
            f"https://portaldatransparencia.gov.br/download-de-dados/servidores/"
            f"{dat_ref}_Aposentados_BACEN"
        ),
        "Pensionistas_BACEN": (
            f"https://portaldatransparencia.gov.br/download-de-dados/servidores/"
            f"{dat_ref}_Pensionistas_BACEN"
        ),
    }

    dict_data = {chave: None for chave in urls}

    target_dir = "data/raw/transparencia"
    dat_ref = datetime.strptime(dat_ref, "%Y%m")

    for chave, base_url in urls.items():
        current_ref = dat_ref

        while True:
            # Constrói a URL com a data atual
            url = base_url.replace(
                base_url.split("/")[-1][:6], current_ref.strftime("%Y%m")
            )
            response = requests.head(url, allow_redirects=True, timeout=600)

            if response.status_code == 200:
                baixar_extrair_transparencia(url, target_dir)
                print(f"Arquivo {url} baixado com sucesso.")
                dict_data[chave] = current_ref.strftime("%Y%m")
                break
            if response.status_code == 403:
                current_ref -= relativedelta(months=1)
                current_ref = current_ref.replace(day=1)
    print(f"Processamento para {dat_ref.strftime('%Y%m')} concluído.")
    return dict_data

In [14]:
baixar_dados_referencia()

AttributeError: module 'datetime' has no attribute 'strptime'

In [16]:
!pip install pandas

  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ------------------- -------------------- 5.5/11.5 MB 33.7 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 35.8 MB/s eta 0:00:00
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)


In [17]:
import datetime
import requests
import zipfile
from io import BytesIO
import pandas as pd

def extrai_servidores_portal_df():
    """
    Função para extrair os dados dos servidores do Portal da Transparência do DF.

    Retorna um DataFrame com os dados dos servidores.

    """
    ano = datetime.datetime.now().year
    url = f"https://www.transparencia.df.gov.br/arquivos/Servidores_Orgao_{ano}.zip"

    resposta = requests.get(url, timeout=100)
    if resposta.status_code != 200:
        print(f"Falha ao baixar o arquivo para o ano {ano}. Tentando o ano anterior...")
        ano -= 1
        url = f"https://www.transparencia.df.gov.br/arquivos/Servidores_Orgao_{ano}.zip"
        resposta = requests.get(url, timeout=100)
    
    if resposta.status_code == 200:
        with zipfile.ZipFile(BytesIO(resposta.content)) as ozip:
            lista_arquivos = ozip.namelist()
            arquivos_csv = [f for f in lista_arquivos if f.endswith(".csv")]
            arquivos_csv.sort(reverse=True)
            if not arquivos_csv:
                print(f"Nenhum arquivo CSV encontrado para o ano {ano}.")
                return None
            csv_mais_recente = arquivos_csv[0]
            ozip.extract(csv_mais_recente)
            print(f"O arquivo CSV mais recente é: {csv_mais_recente}")
            return pd.read_csv(csv_mais_recente, sep=";", encoding="latin1")
    else:
        print(
            f"Falha ao baixar o arquivo para o ano {ano}. Código de status: {resposta.status_code}"
        )
        return None

In [19]:
extrai_servidores_portal_df()

Falha ao baixar o arquivo para o ano 2025. Tentando o ano anterior...
O arquivo CSV mais recente é: Servidores_Orgao_2024_11.csv


C:\Users\14280194645\AppData\Local\Temp\ipykernel_26060\2701334621.py:35: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(csv_mais_recente, sep=";", encoding="latin1")


,NOME,MATRÍCULA,ÓRGAO,CARGO,FUNÇÃO,SÍMBOLO DA FUNÇÃO,SITUAÇÃO FUNCIONAL GERAL,CARREIRA,LOTAÇÃO,DATA DE ADMISSÃO,DATA DE ADMISSÃO - FORMATO NUMÉRICO,SITUAÇÃO,TIPO DE VÍNCULO,MÊS DE REFERÊNCIA,SITUAÇÃO FUNCIONAL ESPECÍFICA
0,ODILAR BOMBASSARO,00251801,SECRETARIA DE ESTADO DE SEGURANCA PUBLICA DO D...,ANALISTA POL PUBL E GEST GOV,NaN,NaN,ATIVO,POLITICAS PUBLICAS E GESTAO GOVERNAMENTAL DO DF,NUCLEO DE MANUTENCAO,17/10/1984,19841017,DESLIGADO,ESTATUTARIO,11,SERVIDOR ESTATUTARIO
1,IVANILDA LOPES DO NASCIMENTO QUEIROZ,00305545,SECRETARIA DE ESTADO DE SEGURANCA PUBLICA DO D...,ANALISTA POL PUBL E GEST GOV,NaN,NaN,ATIVO,POLITICAS PUBLICAS E GESTAO GOVERNAMENTAL DO DF,CEDIDOS / A DISPOSICAO,04/07/1989,19890704,DESLIGADO,ESTATUTARIO,11,SERVIDOR ESTATUTARIO
2,JOSE ALVES DA SILVA,00306916,SECRETARIA DE ESTADO DE SEGURANCA PUBLICA DO D...,TECNICO POL PUBL E GEST GOV,SECRETARIO EXECUTIVO,CPC07,ATIVO,POLITICAS PUBLICAS E GESTAO GOVERNAMENTAL DO DF,CONSELHO PENITENCIARIO DO DISTRITO FEDERAL,26/07/1989,19890726,NORMAL,ESTATUTARIO,11,SERVIDOR ESTATUTARIO
3,JOAO JOSE DE OLIVEIRA,00311405,SECRETARIA DE ESTADO DE SEGURANCA PUBLICA DO D...,ANALISTA POL PUBL E GEST GOV,NaN,NaN,ATIVO,POLITICAS PUBLICAS E GESTAO GOVERNAMENTAL DO DF,GERENCIA DE DOCUMENTACAO ADMINISTRATIVA,18/09/1989,19890918,NORMAL,ESTATUTARIO,11,SERVIDOR ESTATUTARIO
4,GENIVAL DA SILVA PINTO DOS SANTOS,00317071,SECRETARIA DE ESTADO DE SEGURANCA PUBLICA DO D...,ANALISTA POL PUBL E GEST GOV,NaN,NaN,ATIVO,POLITICAS PUBLICAS E GESTAO GOVERNAMENTAL DO DF,NUCLEO DE ALMOXARIFADO,11/01/1990,19900111,DESLIGADO,ESTATUTARIO,11,SERVIDOR ESTATUTARIO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199399,THEO NARDES LACERDA DE CARVALHO,17096499,FUNDO CAPITALIZADO DOS SERVIDORES DO DISTRITO ...,ANA.POL.PUB.GEST.EDUCACIONAL,NaN,NaN,PENSAO,POLITICAS PUBLICAS E GESTAO EDUCACIONAL,SEE - PENSIONISTAS,04/09/2023,20230904,APOSENTADO OU PENSIONISTA,ESTATUTARIO,11,PENSIONISTA ESTAT. INTEGRAL
199400,WALISSON GUIMARAES PEREIRA,17098319,FUNDO CAPITALIZADO DOS SERVIDORES DO DISTRITO ...,PROFESSOR DE EDUC. BASICA,NaN,NaN,PENSAO,MAGISTERIO PUBLICO DO DF,SEE - PENSIONISTAS,12/03/2024,20240312,APOSENTADO OU PENSIONISTA,ESTATUTARIO,11,PENSIONISTA MAGISTERIO (FEDF)
199401,PAULO EVERTON LEITE,17113180,FUNDO CAPITALIZADO DOS SERVIDORES DO DISTRITO ...,ANA.POL.PUB.GEST.EDUCACIONAL,NaN,NaN,PENSAO,POLITICAS PUBLICAS E GESTAO EDUCACIONAL,SEE - PENSIONISTAS,15/07/2024,20240715,APOSENTADO OU PENSIONISTA,ESTATUTARIO,11,PENSIONISTA ESTAT. INTEGRAL
199402,HENRIQUE MARQUES FEITOSA,17120039,FUNDO CAPITALIZADO DOS SERVIDORES DO DISTRITO ...,PROFESSOR DE EDUC. BASICA,NaN,NaN,PENSAO,MAGISTERIO PUBLICO DO DF,SEE - PENSIONISTAS,01/08/2024,20240801,APOSENTADO OU PENSIONISTA,ESTATUTARIO,11,PENSIONISTA MAGISTERIO (FEDF)
